<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-17 13:31:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-17 13:31:59 (75.5 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-17 13:32:03--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-09-17 13:32:03 (120 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 ÛÏYou see the devastation &amp; itÛªs shocking.ÛÏ  Firefighters continue to battle flames in California. benstracy reports Û_ 

---

target:0 (not a disaster)
Text
 Know them recognize them......then obliterate them! 
#gym #gymflow #gymtime #team #assassinsÛ_ https://t.co/mUHj8CbdQb 

---

target:1 (real disaster)
Text
 There might be casualties tomorrow 

---

target:0 (not a disaster)
Text
 [Jax(MK2)] Stage Fatality:UUD+LK (Close) 

---

target:0 (not a disaster)
Text
 The trouble in one of Buffett's favorite sectors http://t.co/J4dqPFLMkR 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 Since 1940 the year Angela Sanders was born roundhouse kick related deaths have increased 13000 percent.       
 Vectorized version:
 [[ 216    1    2  183    1 8832   23 4071 8962 1651 1779  401   24 2424
  3220]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Full Episode:  WN 08/02/15: California Wild Fires Force 12000 to Evacuate #Worldnews http://t.co/9ikhdyHVnC         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00468328, -0.03727007,  0.00787397, ...,  0.0223943 ,
          0.01546928, -0.00011215],
        [ 0.02634409,  0.01328006, -0.04486792, ..., -0.01671238,
         -0.00822606, -0.01103531],
        [-0.04000672,  0.0295484 , -0.04203892, ..., -0.02245597,
          0.01559571,  0.03321607],
        ...,
        [-0.0210685 ,  0.02845024,  0.03789176, ..., -0.00718107,
         -0.03286295,  0.01113621],
        [ 0.03644529,  0.04856441,  0.01887736, ..., -0.00500678,
         -0.00342846,  0.00452933],
        [ 0.03644529,  0.04856441,  0.01887736, ..., -0.00500678,
         -0.00342846,  0.00452933]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('Full Episode:  WN 08/02/15: California Wild Fires Force 12000 to Evacuate #Worldnews http://t.co/9ikhdyHVnC',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00468328, -0.03727007,  0.00787397,  0.0009643 ,  0.0064293 ,
        -0.01483298, -0.01479328, -0.0369871 , -0.03292201, -0.02929345,
         0.00316833, -0.03150641,  0.00854634,  0.00755223,  0.03575614,
         0.00745139, -0.04907334,  0.01749172, -0.02339159,  0.01237734,
         0.0234197 , -0.03471075,  0.01376822,  0.02576974, -0.03978347,
         0.03077407, -0.03221138, -0.03371272, -0.03768711, -0.0111616 ,
        -0.02534977,  0.02239945,  0.04922705, -0.04026735, -0.00951656,
         0.03763033, -0.03377156,  0.03935386, -0.00886031, -0.01249343,
         0.03791152,  0.04941962,  0.00893483,  0.01845545, -0.04168869,
         0.00841026,  0.00973098, -0.02073435,  0.03525568,  0.04736011,
         0.03518182,  0.04783772, -0.02103909, -0.01252068,  0.03477401,
        -0.0413745 ,  0.04609637,  0

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220917-133204
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.6107 - accuracy: 0.6998 - val_loss: 0.5364 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 7s 33ms/step - loss: 0.4417 - accuracy: 0.8190 - val_loss: 0.4671 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 8s 35ms/step - loss: 0.3472 - accuracy: 0.8605 - val_loss: 0.4605 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.2849 - accuracy: 0.8915 - val_loss: 0.4668 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.2377 - accuracy: 0.9104 - val_loss: 0.4792 - val_accuracy: 0.7848


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4792 - accuracy: 0.7848


[0.4791876971721649, 0.7847769260406494]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.39564425],
       [0.8105885 ],
       [0.9969462 ],
       [0.12962073],
       [0.10021025],
       [0.93997645],
       [0.925071  ],
       [0.9932852 ],
       [0.9691092 ],
       [0.2850809 ]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7847769028871391,
 'precision': 0.7895833902086796,
 'recall': 0.7847769028871391,
 'f1': 0.7817499108999565}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False, False])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks

###Model 2 : LSTM

In [46]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [47]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [49]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220917-134303
Epoch 1/5
215/215 [==============================] - 10s 35ms/step - loss: 0.2195 - accuracy: 0.9242 - val_loss: 0.6026 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 7s 33ms/step - loss: 0.1562 - accuracy: 0.9428 - val_loss: 0.6219 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 6s 28ms/step - loss: 0.1298 - accuracy: 0.9498 - val_loss: 0.6671 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.1054 - accuracy: 0.9572 - val_loss: 0.7808 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 11s 52ms/step - loss: 0.0851 - accuracy: 0.9664 - val_loss: 0.9206 - val_accuracy: 0.7730


In [50]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[6.9189370e-03],
        [6.7877865e-01],
        [9.9972153e-01],
        [6.2096566e-02],
        [4.2441487e-04],
        [9.9640888e-01],
        [7.7525228e-01],
        [9.9980128e-01],
        [9.9970257e-01],
        [5.2509385e-01]], dtype=float32))

In [52]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7729658792650919,
 'precision': 0.775649278370019,
 'recall': 0.7729658792650919,
 'f1': 0.7704381702262061}

In [56]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}